In [8]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 



In [9]:
#create a set of random latitudes and longitudes as a combination.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [10]:
#add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [1]:
from citipy import citipy

ModuleNotFoundError: No module named 'citipy'